!pip install -q --upgrade git+https://github.com/SafeGraphInc/safegraph_py

# Load, test, help for SGPY
https://github.com/SafeGraphInc/safegraph_py?tab=readme-ov-file

In [1]:
from safegraph_py_functions import safegraph_py_functions as sgpy
import pandas as pd
import numpy as np
import os


In [2]:
sgpy.test_me_sgpy()
sgpy.version_sgpy()
sgpy.help_sgpy()

Hello World
safegraph_py v1.1.0


 ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
.d8888b.            .d888          .d8888b.                            888           8888888b.           888    888                             888      d8b 888                                       
d88P  Y88b          d88P"          d88P  Y88b                          888           888   Y88b          888    888                             888      Y8P 888                                       
Y88b.               888            888    888                          888           888    888          888    888                             888          888                                       
 "Y888b.    8888b.  888888 .d88b.  888        888d888 8888b.  88888b.  88888b.       888   d88P 888  888 888888 88888b.   .d88b.  88888b.       888      888 88888b.  

# My Functions

In [3]:
# My functions

def list_subsubfolders(root_dir):
    subsubfolders = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        level = dirpath.replace(root_dir, '').count(os.sep)
        if level == 2:  # Change this number to adjust the depth
            subsubfolders.extend([os.path.join(dirpath, d) for d in dirnames])
            # subsubfolders.extend(dirnames)
    subsubfolders.sort()
    return subsubfolders


# Directories
Intoduction to the filesystem of the data.

### Root Directory
Directory for all data files.

In [4]:
root_dir = '/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/'
!ls {root_dir}

core-places-US-Nov2020-present
core-places-US-Pre-Nov-2020
neighborhood-patterns
Social-Distancing-Metrics
Weekly-Places-Patterns-for-data-backfill-for-Dec-2020-and-Onward
Weekly-Places-Patterns-for-data-from-2018-12-31-to-2020-06-15
Weekly-Places-Patterns-for-data-from-2020-06-15-to-2020-11-30
Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present


### Points of Interest (POI) Directory
Directory for POI information (core-places). We used POI data from 2022 for all weeks, assuming SafeGraph ID numbers are not duplicated.

In [5]:
poi_dir = root_dir + 'core-places-US-Nov2020-present/core_poi/'
!ls {poi_dir}

2020  2021  2022


Each year has several subfolders containing 5-7 parts.

In [6]:
poi_dir_detailed = root_dir + 'core-places-US-Nov2020-present/core_poi/' + '2022/01/08/02/'
!ls {poi_dir_detailed}

core_poi-part1.csv.gz  core_poi-part4.csv.gz  core_poi-part7.csv.gz
core_poi-part2.csv.gz  core_poi-part5.csv.gz  _SUCCESS
core_poi-part3.csv.gz  core_poi-part6.csv.gz


### Weekly Patterns Directory
Directory for weekly visits for all areas. Use backfill data for 01/2018-11/2020 and present data for 12/2020-07/2021.

In [7]:
weekly_dir_pre2021 = (root_dir + 
                      'Weekly-Places-Patterns-for-data-backfill-for-Dec-2020-and-Onward/patterns_backfill/'
                      '2020/12/14/21/'
                     )
weekly_dir_2021 = (root_dir +
                   'Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/'
                  )

!ls {weekly_dir_pre2021}
!ls {weekly_dir_2021}

2018  2019  2020
2020  2021


Each year contains subfolders for weekly data, each containing 4-6 parts.

In [8]:
weekly_dir_detailed = (root_dir +
                       'Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/'
                       '2021/01/06/19'
                      )
!ls {weekly_dir_detailed}

patterns-part1.csv.gz  patterns-part3.csv.gz  _SUCCESS
patterns-part2.csv.gz  patterns-part4.csv.gz


### Metadata and normalization stats directory

Contains information about number of POIs, core_places used, number of total visits, and number of devices.

In [9]:
meta_dir = (
    '/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/'
    'Weekly-Places-Patterns-for-data-backfill-for-Dec-2020-and-Onward/release_metadata_backfill/2020/12/14/21/2020/01/06/'
)

!ls {meta_dir}

release_metadata.csv  _SUCCESS


In [14]:
meta_data = pd.read_csv(meta_dir + 'release_metadata.csv')
print(meta_data)

       metadata_description metadata_value
0  core_places_version_used        11-2020
1                 total_poi        4155518
2         total_branded_poi         916102


In [12]:
norm_dir = (root_dir +
            'Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/'
            'normalization_stats/'
           )
!ls {norm_dir}

2020  2021


In [13]:
norm_data = pd.read_csv(norm_dir + '2021/01/06/19/' + 'normalization_stats.csv')
print(norm_data)

     year  month  day      region  total_visits  total_devices_seen  \
0    2020     12   28  ALL_STATES      46985087            16533681   
1    2020     12   28          ak         51622               21432   
2    2020     12   28          al       1254947              398936   
3    2020     12   28          ar        671434              225731   
4    2020     12   28          as            16                  15   
..    ...    ...  ...         ...           ...                 ...   
394  2021      1    3          vt         47484               22392   
395  2021      1    3          wa        630655              292087   
396  2021      1    3          wi        694717              308181   
397  2021      1    3          wv        221301               97826   
398  2021      1    3          wy         58754               26930   

     total_home_visits  total_home_visitors  
0             20998638             13325194  
1                23567                17001  
2        

# Data Processing

### 1. Core POI Data
Get a list of POIs for POI Master List and for merging with weekly patterns. For a given year, list the directories for the core poi data, to view dates available.

In [4]:
# Use this for 12/2020-07/2021
root_directory = ('/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/'
                  'core-places-US-Nov2020-present/core_poi/')

# Use this for 01/2018-11/2020
# root_directory = ('/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/'
#                   'core-places-US-Pre-Nov-2020/')


year = '2020'
subsubfolders = list_subsubfolders(root_directory + year)
for folder in subsubfolders:
    print(folder)

/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/11/06/12
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/12/04/04


Get core poi data and save to a csv file. If not all columns are needed, 'usecol=['col_name', 'col_name', ...]'. If not all dates are needed, adjust range in for loop.

In [ ]:
df_list = []
for folder in subsubfolders[:1]:
    core_poi_filepath = folder
    df = sgpy.read_core_folder(core_poi_filepath,
                               # usecols=['placekey', 'safegraph_place_id']
                               )
    df_list.append(df)
poi_df = pd.concat(df_list)
poi_df = poi_df.drop_duplicates()
poi_df.to_csv(f'poi_all_{year}.csv', index=False)
print(poi_df.columns)
poi_df.head()

/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/11/06/12/core_poi-part1.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/11/06/12/core_poi-part5.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/11/06/12/core_poi-part2.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/11/06/12/core_poi-part3.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/core-places-US-Nov2020-present/core_poi/2020/11/06/12/core_poi-part4.csv.gz


Alternatively, load previously saved data.

TODO - Fix warning about mixed types.

In [ ]:
year = '2021'
filename = f'poi_all_{year}.csv'
poi_df = pd.read_csv(filename)
print(poi_df.columns)
poi_df.head()

Filter core poi data by state and columns (must keep 'safegraph_place_id') and save to .csv file. This drastically reduces size of file.

In [132]:
# Filter poi data
state = 'TX'
columns = ['safegraph_place_id',
           'location_name',
           'naics_code',
           'latitude',
           'longitude']
filtered_poi_df = poi_df[poi_df['region'] == state]
filtered_poi_df = filtered_poi_df[columns]
filtered_poi_df.to_csv(f'poi_{state}_{year}.csv', index=False)
print(filtered_poi_df.shape)
filtered_poi_df.head()

(447853, 5)


,safegraph_place_id,location_name,naics_code,latitude,longitude
40,sg:02b53f54944c4b1a8d00d25a2121594d,Pigtails & Crewcuts,812112.0,30.527127,-97.814557
54,sg:0361b3394dbc4d46a50ef4d7561208ca,8th Street Boutique,448120.0,32.350617,-95.299808
60,sg:03e26b5b4f3b4b2b851d5021ccc9c51e,Navy Federal Credit Union,522130.0,31.772850,-106.399851
101,sg:07e6c5acf7b94403aab792d62d1cf5c0,Sprouts Farmers Market,445110.0,32.811066,-96.772675
124,sg:0a0f61967d0949bcaf7b37fadcb66fd9,Boost Mobile,517312.0,33.548494,-101.849862


Alternatively, load a previously saved filtered file.

In [5]:
year = 2021
filename = f'poi_TX_{year}.csv'
poi_df = pd.read_csv(filename)
print(poi_df.columns)
poi_df.head()

Index(['safegraph_place_id', 'location_name', 'naics_code', 'latitude',
       'longitude'],
      dtype='object')


/glade/derecho/scratch/mbutle15/tmp/ipykernel_38482/2776424985.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  poi_df = pd.read_csv(filename)


,safegraph_place_id,location_name,naics_code,latitude,longitude
0,sg:02b53f54944c4b1a8d00d25a2121594d,Pigtails & Crewcuts,812112.0,30.529074,-97.814428
1,sg:0361b3394dbc4d46a50ef4d7561208ca,8th Street Boutique,448120.0,32.350617,-95.299808
2,sg:07e6c5acf7b94403aab792d62d1cf5c0,Sprouts Farmers Market,445110.0,32.811066,-96.772675
3,sg:0a0f61967d0949bcaf7b37fadcb66fd9,Boost Mobile,517312.0,33.548494,-101.849862
4,sg:0bfabeed776044d88151892984c30421,Citywide Prayer Band Outreach Ministry,813110.0,29.868737,-95.488427


In [54]:
poi_df.head()

,safegraph_place_id,location_name,naics_code,latitude,longitude
0,sg:02b53f54944c4b1a8d00d25a2121594d,Pigtails & Crewcuts,812112.0,30.529074,-97.814428
1,sg:0361b3394dbc4d46a50ef4d7561208ca,8th Street Boutique,448120.0,32.350617,-95.299808
2,sg:07e6c5acf7b94403aab792d62d1cf5c0,Sprouts Farmers Market,445110.0,32.811066,-96.772675
3,sg:0a0f61967d0949bcaf7b37fadcb66fd9,Boost Mobile,517312.0,33.548494,-101.849862
4,sg:0bfabeed776044d88151892984c30421,Citywide Prayer Band Outreach Ministry,813110.0,29.868737,-95.488427


### 2. Weekly Patterns Data
Get weekly patterns data with filtering for needed data.

List directories for weekly patterns data to view dates available. The list is returned sorted alphabetically.

In [6]:
year = '2021'
# patterns_root_folder = r'/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-backfill-for-Dec-2020-and-Onward/patterns_backfill/2020/12/14/21'
patterns_root_folder = r'/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021'

patterns_folders = list_subsubfolders(patterns_root_folder)
for i, folder in enumerate(patterns_folders):
    print(i)
    print(folder)

0
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/06/19
1
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/13/19
2
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/20/19
3
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/27/21
4
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/02/03/21
5
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/02/10/19
6
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Week

In [28]:
print(len(patterns_folders))

28


In [29]:
# Read one file for column names
patterns_onefile = sgpy.read_pattern_single('/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/07/14/17/patterns-part1.csv.gz')
print(patterns_onefile.columns)
patterns_onefile.head()

Index(['placekey', 'parent_placekey', 'location_name', 'street_address',
       'city', 'region', 'postal_code', 'iso_country_code',
       'safegraph_brand_ids', 'brands', 'date_range_start', 'date_range_end',
       'raw_visit_counts', 'raw_visitor_counts', 'visits_by_day',
       'visits_by_each_hour', 'poi_cbg', 'visitor_home_cbgs',
       'visitor_home_aggregation', 'visitor_daytime_cbgs',
       'visitor_country_of_origin', 'distance_from_home', 'median_dwell',
       'bucketed_dwell_times', 'related_same_day_brand',
       'related_same_week_brand', 'device_type'],
      dtype='object')


,placekey,parent_placekey,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,...,visitor_home_cbgs,visitor_home_aggregation,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type
0,223-222@8gd-nfm-gff,NaN,Doug's Auto Collision Center,2985 E Oglethorpe Hwy,Hinesville,GA,31313,US,NaN,NaN,...,"{""131790104001"":4}","{""13179010400"":4}",{},"{""US"":4}",NaN,4.5,"{""<5"":2,""5-10"":2,""11-20"":0,""21-60"":0,""61-120"":...",{},{},"{""android"":0,""ios"":4}"
1,zzz-222@63j-f68-6rk,NaN,Activity Center Park,185 Hampton Rd,Centerville,OH,45459,US,NaN,NaN,...,"{""391130404031"":4,""391093651011"":4,""3911304030...","{""39113040401"":6,""39113040306"":6,""39113040305""...","{""391130403063"":5,""390572201001"":4,""3911309030...","{""US"":77}",6750.0,46.0,"{""<5"":1,""5-10"":12,""11-20"":8,""21-60"":51,""61-120...","{""Speedway"":14,""McDonald's"":9,""Chick-fil-A"":7,...","{""McDonald's"":32,""Speedway"":28,""Kroger Fuel Ce...","{""android"":35,""ios"":38}"
2,22b-222@5s8-cjs-rx5,NaN,Rainbow Child Development Center,605 Como Ave,Saint Paul,MN,55103,US,NaN,NaN,...,{},{},{},{},NaN,12.0,"{""<5"":0,""5-10"":0,""11-20"":1,""21-60"":0,""61-120"":...",{},{},"{""android"":4,""ios"":0}"
3,223-222@5pt-52d-py9,NaN,Panda Garden,2801 W Kingshighway Ste 1,Paragould,AR,72450,US,NaN,NaN,...,"{""050554802002"":7,""050554808012"":5,""0505548050...","{""05055480801"":9,""05055480500"":8,""05055480200""...","{""050554805003"":6,""050554802002"":4,""0505548040...","{""US"":53}",6401.0,22.0,"{""<5"":4,""5-10"":16,""11-20"":7,""21-60"":24,""61-120...","{""Walmart"":30,""Kum & Go"":16,""Sonic"":14,""PetSma...","{""Walmart"":80,""Sonic"":57,""Dollar General"":44,""...","{""android"":20,""ios"":29}"
4,222-222@5pw-6cg-fzz,NaN,Senior Lifestyle,8300 S Cottage Grove Ave,Chicago,IL,60619,US,SG_BRAND_0268ea466434672b,Senior Lifestyle,...,{},{},{},"{""US"":4}",NaN,8.0,"{""<5"":1,""5-10"":1,""11-20"":1,""21-60"":0,""61-120"":...",{},{},"{""android"":5,""ios"":4}"


Get weekly visits for a specific number of weeks (alternatively, adjust range in for loop). Specify columns (must keep 'safegraph_place_id' and 'date_range_end') and FIPS code for desired region. This drops any NA values for POI cbgs and creates a list of dataframes, each list entry contains one week.

In [7]:

columns = ['poi_cbg',
           'visitor_home_cbgs',
           'raw_visit_counts',
           'raw_visitor_counts',
           'safegraph_place_id',
           'date_range_end',
           'device_type',
           'distance_from_home']

FIPS = '48201'

patterns_df_list = []
filtered_patterns_df_list = []

# Change index for subcategory of weeks
for folder in patterns_folders:
    patterns_df = sgpy.read_pattern_multi(folder, usecols=columns)
    patterns_df = patterns_df.dropna(subset=['poi_cbg'])
    filtered_patterns_df = patterns_df[patterns_df['poi_cbg'].str.startswith(FIPS)]
    patterns_df_list.append(patterns_df)
    filtered_patterns_df_list.append(filtered_patterns_df)

/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/06/19/patterns-part3.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/06/19/patterns-part4.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/06/19/patterns-part2.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/06/19/patterns-part1.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/patterns/2021/01/13/19/patterns-part3.csv.gz
/glade/work/daneta/SafeGraph/SafeGraphData_BatchDownload_on_2022_2_10/Weekly-Places-Patterns-for-data-from-2020-11-30-to-Present/p

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [31]:
# Checking weekly patterns dataframes
print(len(filtered_patterns_df_list))
filtered_patterns_df_list[0].head()

26


,safegraph_place_id,date_range_end,raw_visit_counts,raw_visitor_counts,poi_cbg,visitor_home_cbgs,distance_from_home,device_type
23,sg:01e5cf1972a9453ab650d070d4edb3ae,2021-01-04T00:00:00-06:00,1,1,482012205002,{},NaN,"{""android"":0,""ios"":0}"
82,sg:06be9d3b7b484bdf962683759313bd66,2021-01-04T00:00:00-06:00,2,2,482014311022,{},NaN,"{""android"":0,""ios"":0}"
113,sg:09368ac62c134897b98ba739cda14fee,2021-01-04T00:00:00-06:00,12,7,482015546002,"{""482015545022"":4}",2901.0,"{""android"":4,""ios"":4}"
196,sg:118be774dd0847d98d91dfe48930d7d6,2021-01-04T00:00:00-06:00,54,33,482014222003,"{""482014236003"":4,""482014222001"":4,""4815767110...",13497.0,"{""android"":15,""ios"":18}"
324,sg:1e81f3e133184602a32cbffd26d8bbc5,2021-01-04T00:00:00-06:00,6,4,482014228002,{},NaN,"{""android"":0,""ios"":0}"


### 3. Merge POI and weekly patterns dataframes
Merge POI and weekly patterns to get .csv files for each week. Also expand JSON columns here.

Merge the core poi dataframe and patterns dataframe. A list of merged dataframes (one list entry for each week) is created. This also collect the date range end for each week and stores it in end_dates.

Uses filtered_patterns_df_list from previous cell (might want to rethink that).

In [10]:
poi_df = pd.read_csv('poi_TX_2020_new.csv')
merged_df_list = []
end_dates = []
for patterns_df in filtered_patterns_df_list:
    date = patterns_df['date_range_end'].iloc[0][:10]
    print(date)
    end_dates.append(date)
    df = sgpy.merge_core_pattern(poi_df, patterns_df)
    # df = df[['location_name', 'safegraph_place_id', 'naics_code', 'poi_cbg', 'longitude', 'latitude']]
    merged_df_list.append(df)
print(merged_df_list[0].head())

2021-01-04
2021-01-11
2021-01-18
2021-01-25
                    safegraph_place_id        location_name  naics_code  \
0  sg:18fc341634df48118ba5daeb3fb276fc  Honey Bee Beautiful    812112.0   
1  sg:192d7845bc8945dfa63a25bbf35574e2            Menchie's    722515.0   
2  sg:20421578b6c041ed8569e517fe8e449c     Memorial Hermann    621111.0   
3  sg:2869bfbf54594ead8d3afe118537333d   Sarpino's Pizzeria    722511.0   
4  sg:2eaa6167052d41c29c321153df8bb802        Discount Tire    441310.0   

    latitude  longitude             date_range_end  raw_visit_counts  \
0  30.073127 -95.526225  2021-01-04T00:00:00-06:00                 1   
1  30.012920 -95.262418  2021-01-04T00:00:00-06:00               125   
2  29.776253 -95.746255  2021-01-04T00:00:00-06:00                66   
3  29.923806 -95.556749  2021-01-04T00:00:00-06:00                24   
4  29.799468 -95.515912  2021-01-04T00:00:00-06:00                21   

   raw_visitor_counts       poi_cbg  \
0                   1  4820155490

Run to save weekly visits without expanding JSON

In [11]:
for i, df in enumerate(merged_df_list):
    date = end_dates[i]
    filename = f'unexpanded_{date}.csv'
    df.to_csv(filename, index=False)

### 4. Unpacking JSON data
This unpacks the visitor cbg column to provide counts for each visitor cbg. It is time intensive (possible filtering before).

In [9]:
# Unpack JSON data
# Time intensive
unpacked_df_list = []
for i, merged_df in enumerate(merged_df_list):
    date = end_dates[i]
    print(date)
    df = sgpy.unpack_json_and_merge_fast(merged_df)
    # df = df[['naics_code', 'poi_cbg', 'visitor_home_cbgs_key', 'visitor_home_cbgs_value', 'longitude', 'latitude']]
    # df = df.rename(columns={'visitor_home_cbgs_key': 'visitor_cbg', 'visitor_home_cbgs_value': 'count'})
    df.to_csv(f'weekly_patterns_{date}.csv', index=False)
    


2021-01-04
2021-01-11
2021-01-18
2021-01-25
